In [81]:
from pysentimiento import create_analyzer
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
# from pysentimiento.preprocessing import preprocess_tweet # using github preprocess instead
import preprocessor as p
import spacy
import tweepy
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

b_analyzer_sentiment = create_analyzer(task="sentiment", lang="en")
b_analyzer_emotion = create_analyzer(task="emotion", lang="en")
b_analyzer_hate_speech = create_analyzer(task="hate_speech", lang="en")
v_analyzer_sentiment = SentimentIntensityAnalyzer()
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG)
TweetTokenizer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

loading configuration file https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis/resolve/main/config.json from cache at C:\Users\Joe Wilson/.cache\huggingface\transformers\c246eed05359b1a49c45955b0265b488e35b0cbd2628e3ead7dd54c8815162ee.a2dff24b4e0a884c6d58a09968c5b68e7391e749eb698ad92541818d420fd01b
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others

Realtime Tweet Sentiment Analyzer using BERTWEET engine and customized ML

In [82]:
def tweet_sentiment_analyzer(text):
    
    temp = b_analyzer_sentiment.predict(text)
    bert_sentiment_label = temp.output # output = neg, pos, neu label
    bert_sentiment_score = temp.probas # probas = percentage score
    NEG = round(bert_sentiment_score.get('POS'), 2)
    POS = round(bert_sentiment_score.get('NEG'), 2)
    NEU = round(bert_sentiment_score.get('NEU'), 2)
    score_probability = max(NEG, POS, NEU)
    bert_sentiment_list = [bert_sentiment_label, score_probability]
    
    return(bert_sentiment_list)

Realtime Tweet Tokenizer using nltk's tweet tokenizer

In [83]:
def tweet_tokenization(clean_tweet_text):
    
    token = TweetTokenizer()
    temp_tokens = token.tokenize(clean_tweet_text)  
    nltk_tokens = []
    for w in temp_tokens:
         if w not in stop_words:
             if len(w) > 2:
                 nltk_tokens.append(w)
                 
    return(nltk_tokens)

In [86]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')

query = "missouri schools -is:retweet lang:en"
response = client.search_recent_tweets(query=query,tweet_fields=['attachments','author_id','context_annotations','conversation_id','created_at','entities','geo,id','in_reply_to_user_id','lang','possibly_sensitive','public_metrics','referenced_tweets','reply_settings','source','text','withheld'],user_fields=['created_at','description','entities,id','location','name','pinned_tweet_id','profile_image_url','protected,public_metrics','url','username','verified','withheld'],expansions=['attachments.poll_ids','attachments.media_keys','author_id','geo.place_id','in_reply_to_user_id','referenced_tweets.id','entities.mentions.username','referenced_tweets.id.author_id'],media_fields=['duration_ms','height','media_key', 'preview_image_url','promoted_metrics','public_metrics','type,url'],place_fields=['contained_within,country','country_code','full_name','geo,id','name','place_type'],poll_fields=['duration_minutes','end_datetime','id','options','voting_status'],max_results=10)  
for tweet in response.data: 
    # print("orginal tweet: " +tweet.text)
    print('----------------------------------------------------------------------------------------')
    clean_tweet_text = p.clean(tweet.text)
    clean_tweet_text = clean_tweet_text.replace("&amp", "")  
    tweet_sentiment = tweet_sentiment_analyzer(tweet.text)
    nltk_tokens = tweet_tokenization(clean_tweet_text) 
    print(tweet.text)
    print(clean_tweet_text)
    print(nltk_tokens)
    print(tweet_sentiment)
    print(tweet.data)
    print('------------------------------------------------------------------------------------------')
    


----------------------------------------------------------------------------------------
@GovParsonMO Please follow the lead of Governor @GlennYoungkin and issue an EO preventing schools from applying mask mandates. Please see EO 2 by Youngkin for an example of what Missouri desperately needs. I'm sick of my 2 and 4 year old having to wear a mask to school.
Please follow the lead of Governor and issue an EO preventing schools from applying mask mandates. Please see EO 2 by Youngkin for an example of what Missouri desperately needs. I'm sick of my 2 and 4 year old having to wear a mask to school.
['Please', 'follow', 'lead', 'Governor', 'issue', 'preventing', 'schools', 'applying', 'mask', 'mandates', 'Please', 'see', 'Youngkin', 'example', 'Missouri', 'desperately', 'needs', "I'm", 'sick', 'year', 'old', 'wear', 'mask', 'school']
['NEG', 0.86]
{'created_at': '2022-01-20T18:42:46.000Z', 'entities': {'mentions': [{'start': 0, 'end': 12, 'username': 'GovParsonMO', 'id': '10036723207818076

In [79]:
print('hi')

hi
